In [1]:
import pandas as pd
import numpy as np
import os
import time
import cv2
import matplotlib.pyplot as plt
import pickle
from tqdm import tqdm
import random
from sklearn.preprocessing import  OneHotEncoder

In [2]:
from keras.models import Model
from keras.layers import Conv2D, MaxPooling2D, Input, Dense
from keras.layers import Flatten, Activation, Concatenate
from keras.losses import categorical_crossentropy
from keras.optimizers import adam
import keras.backend as K

/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [3]:
FOLDER = '../../../data_handson/kuzushiji/'
IMAGES = FOLDER + 'train_images/'
CHARS = FOLDER + 'chars/'
MODELS = FOLDER + 'models/'
print(os.listdir(FOLDER))

['prior_boxes_ssd300.pkl', '.DS_Store', 'chars', 'unicode_translation.csv', 'models', 'image', 'train.csv', 'gt_pascal.pkl', 'train_images', 'sample_submission.csv']


In [4]:
from ssd_utils.ssd_cnn import SSD_CNN
from ssd_utils.ssd_box import DefaultBox, BBoxUtility
from ssd_utils.ssd_gen import Generator
from ssd_utils.ssd_training import MultiboxLoss

In [5]:
df_train = pd.read_csv(FOLDER + 'train.csv')
df_sub = pd.read_csv(FOLDER + 'sample_submission.csv')
unicode_map = {codepoint: char for codepoint, char in pd.read_csv(FOLDER + 'unicode_translation.csv').values}

ohe = OneHotEncoder()
ohe.fit(pd.Series(list(unicode_map.keys())).unique().reshape(-1,1))

OneHotEncoder(categorical_features=None, categories=None, drop=None,
              dtype=<class 'numpy.float64'>, handle_unknown='error',
              n_values=None, sparse=True)

In [6]:
df_sub_idx = df_sub.set_index("image_id")
df_train_idx = df_train.set_index("image_id")
idx_train = df_train['image_id']

In [7]:
num_classes = len(unicode_map)

In [8]:
MBL = MultiboxLoss(num_classes)
DB = DefaultBox((1000,1000,1), 10, 100, [2,3], [0.1, 0.1, 0.2, 0.2]) 
DB.compute_output_shape(np.array([1,10,20]))
# var = K.ones(shape=( 100, 100))
# priors = K.eval(DB(var))

(1, 1200, 8)

In [9]:
priors = pickle.load(open(FOLDER + 'prior_boxes_ssd300.pkl', 'rb'))
BBU = BBoxUtility(num_classes, priors)

In [10]:
len_idx = len(idx_train)
rand_idx = random.sample(list(idx_train), len_idx)
train_keys = rand_idx[:int(len_idx*0.8)]
val_keys = rand_idx[int(len_idx*0.8):]

In [11]:
BBU = BBoxUtility(num_classes, priors)

df_idx = df_train_idx 
bbox_util = BBU
batch_size = 32
path_prefix = IMAGES
image_size = (300, 300, 1)

gen =  Generator(df_idx = df_train_idx, 
                 bbox_util = BBU,
                 batch_size = batch_size,
                 path_prefix = IMAGES,
                 ohe = ohe,
                 train_keys = train_keys,
                 val_keys = val_keys,
                 image_size=image_size)

In [12]:
ssd_obj = SSD_CNN(num_classes, cnn_size=(100, 100, 1), ssd_size=(300,300,1))
cnn_model = ssd_obj.load(MODELS + 'model.hdf5')

In [13]:
ssd_model = ssd_obj.build_ssd()

In [14]:
ssd_model.compile(optimizer=adam(), loss=categorical_crossentropy)

In [15]:
history = ssd_model.fit_generator(gen.generate(True),  gen.train_batches,
                              1, verbose=1,
                              validation_data=gen.generate(False),
                              nb_val_samples=gen.val_batches)

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:4: UserWarning: The semantics of the Keras 2 argument `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Basically steps_per_epoch = samples_per_epoch/batch_size. Similarly `nb_val_samples`->`validation_steps` and `val_samples`->`steps` arguments have changed. Update your method calls accordingly.
  after removing the cwd from sys.path.
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:4: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<generator..., 3104, 1, verbose=1, validation_data=<generator..., validation_steps=777)`
  after removing the cwd from sys.path.


Epoch 1/1


ValueError: Error when checking target: expected predictions to have shape (11430, 4799) but got array with shape (7308, 4799)

In [ ]:
 df_train_idx.loc[idx_train[0]]['labels']

In [17]:
priors.shape

(7308, 8)

In [18]:
ssd_model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
ssd_input (InputLayer)          (None, 300, 300, 1)  0                                            
__________________________________________________________________________________________________
conv1_1 (Conv2D)                multiple             640         ssd_input[0][0]                  
__________________________________________________________________________________________________
conv1_2 (Conv2D)                multiple             36928       conv1_1[1][0]                    
__________________________________________________________________________________________________
pool1 (MaxPooling2D)            multiple             0           conv1_2[1][0]                    
__________________________________________________________________________________________________
conv2_1 (C

In [20]:
var1 = K.ones(shape=(1, 19, 19))
priors1 = K.eval(DB(var1))
priors1.shape

(1, 2166, 8)

In [21]:
var1 = K.ones(shape=(1, 38, 38))
priors1 = K.eval(DB(var1))
priors1.shape

(1, 8664, 8)

In [24]:
11430 - 2166 + 8664

17928

In [26]:
var1 = K.ones(shape=(1, 76, 76))
priors1 = K.eval(DB(var1))
priors1.shape


(1, 34656, 8)

In [28]:
34656/2

17328.0